In [1]:
from random import seed
import torch
import os
import json
import matplotlib.pyplot as plt
import numpy as np
from torch import optim
from torch.utils.data import DataLoader
from models.Recon_subnetwork import UNetModel, update_ema_params
from models.Seg_subnetwork import SegmentationSubNetwork
from tqdm import tqdm
import torch.nn as nn
from data.dataset_beta_thresh import RealIADTrainDataset, RealIADTestDataset
from math import exp
import torch.nn.functional as F
from models.DDPM import GaussianDiffusionModel, get_beta_schedule
from scipy.ndimage import gaussian_filter
from skimage.measure import label, regionprops
from sklearn.metrics import roc_auc_score,auc,average_precision_score
import pandas as pd
from collections import defaultdict
from train import defaultdict_from_json, train

/home/huy/Project/defect_diffusion/diffad/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
%%writefile args/args1.json
{
  "img_size": [256,256],
  "Batch_Size": 16,
  "EPOCHS": 3000,
  "T": 1000,
  "base_channels": 128,
  "beta_schedule": "linear",
  "loss_type": "l2",
  "diffusion_lr": 1e-4,
  "seg_lr": 1e-5,
  "random_slice": true,
  "weight_decay": 0.0,
  "save_imgs":true,
  "save_vids":false, 
  "dropout":0,
  "attention_resolutions":"32,16,8",
  "num_heads":4,
  "num_head_channels":-1,
  "noise_fn":"gauss",
  "channels":3,
  "data_name":"RealIAD",
  "data_root_path":"datasets",
  "anomaly_source_path":"datasets/dtd",
  "noisier_t_range":600,
  "less_t_range":300,
  "condition_w":1,
  "eval_normal_t":200,
  "eval_noisier_t":400,
  "output_path":"outputs"

}


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# read file from argument
file = "args1.json"
# load the json args
with open(f'./args/{file}', 'r') as f:
    args = json.load(f)
args['arg_num'] = file[4:-5]
args = defaultdict_from_json(args)


real_iad_classes = os.listdir(os.path.join(args["data_root_path"], args['data_name']))

for sub_class in real_iad_classes:   
    print("class", sub_class)
    
    subclass_path = os.path.join(args["data_root_path"], args['data_name'], sub_class)
    
    training_dataset = RealIADTrainDataset(
        subclass_path, sub_class, img_size=args["img_size"], args=args
    )
    testing_dataset = RealIADTestDataset(
        subclass_path, sub_class, img_size=args["img_size"]
    )
    class_type=args['data_name']
    

    print(file, args)     

    data_len = len(testing_dataset) 
    training_dataset_loader = DataLoader(training_dataset, batch_size=args['Batch_Size'],shuffle=True,num_workers=8,pin_memory=True,drop_last=True)
    test_loader = DataLoader(testing_dataset, batch_size=1,shuffle=False, num_workers=4)

    # make arg specific directories
    for i in [f'{args["output_path"]}/model/diff-params-ARGS={args["arg_num"]}/{sub_class}',
            f'{args["output_path"]}/diffusion-training-images/ARGS={args["arg_num"]}/{sub_class}',
                f'{args["output_path"]}/metrics/ARGS={args["arg_num"]}/{sub_class}']:
        try:
            os.makedirs(i)
        except OSError:
            pass


    train(training_dataset_loader, test_loader, args, data_len,sub_class,class_type,device )

class PCB5
args1.json defaultdict(<class 'str'>, {'img_size': [256, 256], 'Batch_Size': 16, 'EPOCHS': 3000, 'T': 1000, 'base_channels': 128, 'beta_schedule': 'linear', 'loss_type': 'l2', 'diffusion_lr': 0.0001, 'seg_lr': 1e-05, 'random_slice': True, 'weight_decay': 0.0, 'save_imgs': True, 'save_vids': False, 'dropout': 0, 'attention_resolutions': '32,16,8', 'num_heads': 4, 'num_head_channels': -1, 'noise_fn': 'gauss', 'channels': 3, 'data_name': 'RealIAD', 'data_root_path': 'datasets', 'anomaly_source_path': 'datasets/dtd', 'noisier_t_range': 600, 'less_t_range': 300, 'condition_w': 1, 'eval_normal_t': 200, 'eval_noisier_t': 400, 'output_path': 'outputs', 'arg_num': '1'})


  0%|          | 0/123 [00:02<?, ?it/s]


OutOfMemoryError: CUDA out of memory. Tried to allocate 512.00 MiB (GPU 0; 3.81 GiB total capacity; 3.15 GiB already allocated; 495.19 MiB free; 3.16 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF